# Reasoning and Knowledge Representation Assignment

Names:

*Selsabeel Asim Ali Elbagir, 20210714*

*Basma Mahmoud Hashem, 20210091*

Section:
*S2*

## Converting to CNF form

In [ ]:
KB = ['∀x (Read(x) → ~Stupid(x))',
      'Read(John) ∧ Wealthy(John)',
      '∀x (Poor(x) → ~Wealthy(x))',
      '∀x Stupid(x) ∨ Smart(x)',
      '∀x ((~Poor(x) ∧ Smart(x)) → Happy(x))',
      '∀x (~Exciting(x) → ~Happy(x))']

### Helper Functions

In [ ]:
import re

In [ ]:
def replace_variable_in_context(formula, variable, replacement):
    # Replace the variable with the replacement character or substring in the specified context
    tokens = tokenize_formula(formula)
    new_tokens = []

    for token in tokens:
        if token == variable:
            # Check if the token satisfies one of the specified conditions
            if (
                (token + ',') in formula or
                (token + ', ') in formula or
                (',' + token) in formula or
                (', ' + token) in formula or
                f'({token})' in formula
            ):
                new_token = replacement
            else:
                new_token = token
        else:
            new_token = token

        new_tokens.append(new_token)

    return "".join(new_tokens)

def tokenize_formula(formula):
    # Tokenize the formula to identify tokens including brackets, commas, and spaces
    tokens = []
    current_token = ""

    for char in formula:
        if char in (',', '(', ')'):
            if current_token:
                tokens.append(current_token.strip())  # Add the trimmed current_token
            current_token = ""
            tokens.append(char)  # Add the bracket or comma as a separate token
        elif char.isspace():
            if current_token:
                tokens.append(current_token.strip())  # Add the trimmed current_token
            current_token = ""
            tokens.append(char)
        else:
            current_token += char

    # Append the last token if it's not empty
    if current_token:
        tokens.append(current_token.strip())

    return tokens

In [ ]:
def replace_operator_in_context(formula_tokens, operator, replacement):
    # Replace the operator with the replacement character or substring in the specified context
    new_tokens = []

    for token in formula_tokens:
        if token == operator:
            new_token = replacement
        else:
            new_token = token

        new_tokens.append(new_token)

    return new_tokens


In [ ]:
def rename_variables(tokens, quantifier_variable, start_index, end_index):
    variable_count = {}
    for i in range(start_index, end_index):
        token = tokens[i]
        if token.isalpha() and token.islower() and token == quantifier_variable:
            if token in variable_count:
                variable_count[token] += 1
            else:
                variable_count[token] = 1
            new_variable = token + str(variable_count[token])
            tokens[i] = new_variable

In [ ]:
def get_surrounding_bracket(formula_tokens, operator):
    left_bracket_index = None
    right_bracket_index = None
    operator_index = None

    # Find the index of the operator
    for i, token in enumerate(formula_tokens):
        if token == operator or token.startswith(operator):
            operator_index = i
            break

    if operator_index is None:
        return None, None  # Operator not found

    # Find the left bracket index
    bracket_counter = 0
    for i in range(operator_index - 1, -1, -1):
        if formula_tokens[i].startswith(')'):
            bracket_counter += 1
        elif formula_tokens[i] == '(':
            if bracket_counter == 0:
                left_bracket_index = i
                break
            else:
                bracket_counter -= 1

    # Find the right bracket index
    bracket_counter = 0
    for i in range(operator_index + 1, len(formula_tokens)):
        if formula_tokens[i].startswith('('):
            bracket_counter += 1
        elif formula_tokens[i] == ')':
            if bracket_counter == 0:
                right_bracket_index = i
                break
            else:
                bracket_counter -= 1

    return left_bracket_index, right_bracket_index


In [ ]:
def find_matching_parenthesis(tokens, start_index):
    bracket_count = 0
    for i in range(start_index, len(tokens)):
        if tokens[i] == '(':
            bracket_count += 1
        elif tokens[i] == ')':
            bracket_count -= 1
            if bracket_count == 0:
                return i
    return -1


### The Ten Steps of CNF Conversion

In [ ]:
def eliminate_implication(formula):
  tokens = tokenize_formula(formula)
  left_bracket_index, right = get_surrounding_bracket(tokens, '→')
  if left_bracket_index is not None and left_bracket_index > 0:
    tokens.insert(left_bracket_index + 1 , '~')  # Insert '~' before the left bracket
  replaced_tokens = replace_operator_in_context(tokens, '→', '∨')
  replaced_formula = "".join(replaced_tokens)
  return replaced_formula


In [ ]:
def move_negation_inwards(formula):
    tokens = tokenize_formula(formula)
    negation_indices = [i for i, token in enumerate(tokens) if token == '~']

    for index in negation_indices:
      if index < len(tokens) - 1:  # Check if negation is not the last token
        next_token = tokens[index + 1]
        if next_token == '(' or next_token.startswith('('):  # Check if negation is followed by a left bracket
          right_bracket_index = find_matching_parenthesis(tokens,index+1)
          for i in range(index,right_bracket_index):
            if tokens[i] == '∧':
              tokens[i] = '∨'
            elif tokens[i] == '∨':
              tokens[i] = '∧'
            elif tokens[i].isalpha() and tokens[i][0].isupper():
              tokens[i] = '~' + tokens[i]

          tokens.pop(index)
          tokens[index+1] = '~' + tokens[index+1]
    replaced_formula = "".join(tokens)
    return replaced_formula


In [ ]:
def remove_double_not(formula):
  tokens = tokenize_formula(formula)
  for i, token in enumerate(tokens):
    if token.startswith('~~'):
      if len(token) == 2:
        tokens.removeAt(i)
      else:
        tokens[i] = token[len('~~'):]
    elif token.startswith('~') and tokens[i+2].startswith('~'):
      i = i+2
      tokens[i] = tokens[i][1:] # removes the ~ before the (
      tokens.pop(i-1) # removes the (
      tokens.pop(i-2)# removes the ~ before it
      tokens.pop(i+1) #removes the ) of the scope


  return "".join(tokens)

In [ ]:
def standardize_variables(formula):
    tokens = tokenize_formula(formula)
    size = len(tokens)
    i = 0
    while i < size:
        if tokens[i].startswith('∀') or tokens[i].startswith('∃'):
            quantifier_variable = tokens[i][1]  # get the variable of the quantifier
            if tokens[i+1] == '(':
              start_scope_index = i + 1
            elif tokens[i+2] == '(' and tokens[i+1] == ' ':
              start_scope_index = i + 2  # gets the index of the (, beginning of our scope

            end_scope_index = find_matching_parenthesis(tokens, start_scope_index)
            # Rename variables within the scope of the quantifier
            rename_variables(tokens, quantifier_variable, end_scope_index, size)
            i = end_scope_index + 1
        else:
            i += 1

    standardized_formula = "".join(tokens)
    return standardized_formula

ex = "∀x (P(x)) ∀x (Q(x, y))"
standardized_formula = standardize_variables(ex)
print("Standardized formula:", standardized_formula)

Standardized formula: ∀x (P(x)) ∀x (Q(x1, y))


In [ ]:
def prenex_form(input_formula):
    tokens = tokenize_formula(input_formula)
    prenex_part = []  # Will store the quantifiers and their variables
    matrix_part = []  # Will store the matrix part of the formula (without quantifiers)
    i = 0
    while i < len(tokens):
        token = tokens[i]
        if token.startswith('∀') or token.startswith('∃'):  # Quantifiers
            quantifier = token
            variables = []
            # Extract variables until the next token is not a variable
            while i + 1 < len(tokens) and tokens[i + 1].isalnum():
                variables.append(tokens[i + 1])
                i += 1
            prenex_part.append((quantifier, variables))
        else:
            matrix_part.append(token)
        i += 1

    # Reconstruct the formula with quantifiers moved to the left
    prenex_formula = ""
    for quantifier, variables in prenex_part:
        prenex_formula += quantifier + ' '.join(variables)
    prenex_formula += ''.join(matrix_part)

    return prenex_formula

formula = "∀x (P(x, y) ∧ ∃y Q(x))"
print("Original formula:", formula)

prenex_formula = prenex_form(formula)
print("Prenex form:", prenex_formula)

Original formula: ∀x (P(x, y) ∧ ∃y Q(x))
Prenex form: ∀x∃y (P(x, y) ∧  Q(x))


In [ ]:
sentences = ["∀x ~(Read(x) → Stupid(x))", "Read(John) ∧ Wealthy(John)", "∀x(Poor(x) → ~Wealthy(x))",
             "∀x(Stupid(x) ∨ Smart(x))", "∀x(~Poor(x) ∧ Smart(x) → Happy(x))", "∀x(~Exciting(x) → ~Happy(x))"]
for sentence in sentences:
  print("Original formula:", sentence)

  eliminated_formula = eliminate_implication(sentence)
  if eliminated_formula != sentence:
    print("After eliminating implication:", eliminated_formula)

  negated_inward_formula = move_negation_inwards(eliminated_formula)
  if negated_inward_formula != eliminated_formula:
    print("After moving negation inwards:", negated_inward_formula)

  removed_formula = remove_double_not(negated_inward_formula)
  if removed_formula != negated_inward_formula:
    print("Removing double not:", removed_formula)

  standardized_formula = standardize_variables(removed_formula)
  if standardized_formula != removed_formula:
    print("After standardizing variables:", standardized_formula)

  prenex_formula = prenex_form(standardized_formula)
  if prenex_formula != standardized_formula:
    print("After prenex form:", prenex_formula)
  print ( "--------------------")


Original formula: ∀x ~(Read(x) → Stupid(x))
After eliminating implication: ∀x ~(~Read(x) ∨ Stupid(x))
After moving negation inwards: ∀x (~~Read(x) ∧ ~Stupid(x))
Removing double not: ∀x (Read(x) ∧ ~Stupid(x))
--------------------
Original formula: Read(John) ∧ Wealthy(John)
--------------------
Original formula: ∀x(Poor(x) → ~Wealthy(x))
After eliminating implication: ∀x(~Poor(x) ∨ ~Wealthy(x))
--------------------
Original formula: ∀x(Stupid(x) ∨ Smart(x))
--------------------
Original formula: ∀x(~Poor(x) ∧ Smart(x) → Happy(x))
After eliminating implication: ∀x(~~Poor(x) ∧ Smart(x) ∨ Happy(x))
Removing double not: ∀x(Poor(x) ∧ Smart(x) ∨ Happy(x))
--------------------
Original formula: ∀x(~Exciting(x) → ~Happy(x))
After eliminating implication: ∀x(~~Exciting(x) ∨ ~Happy(x))
Removing double not: ∀x(Exciting(x) ∨ ~Happy(x))
--------------------




---

---



---





In [ ]:
def skolemize(formula):
    replaced_variables = set()  # Initialize an empty set to store variables that have been replaced
    skolemized_formula = formula.strip()

    while '∃' in skolemized_formula:
        existential_vars = extract_variables(skolemized_formula, '∃')

        existential_vars = [var for var in existential_vars if var not in replaced_variables]

        if not existential_vars:
            break

        if '∀' in skolemized_formula:
            universal_vars = extract_variables(skolemized_formula, '∀')

            skolemized_formula = replace_with_skolem_functions(skolemized_formula, existential_vars, universal_vars)
        else:
            skolemized_formula = replace_with_constants(skolemized_formula, existential_vars)

        replaced_variables.update(existential_vars)

        skolemized_formula = skolemized_formula.strip()

    return skolemized_formula

In [ ]:
def extract_variables(formula, quantifier):
    return [term[1] for term in formula.split() if term[0] == quantifier]

In [ ]:
def replace_with_skolem_functions(formula, existential_vars, universal_vars):
    """
    Replaces existential variables with skolem functions based on the universal variables.
    """
    skolemized_formula = formula
    counter = 1
    for var in existential_vars:
        skolem_func = find_skolem_function(counter, universal_vars)

        skolemized_formula = skolemized_formula.replace('∃{}'.format(var), '')
        skolemized_formula = replace_variable_in_context(skolemized_formula, var, skolem_func)
        counter += 1

    return skolemized_formula

def find_skolem_function(counter, universal_vars):
    """
    Generates a skolem function based on the universal variables and a counter.
    """
    return 'F{}({})'.format(counter, ','.join(universal_vars))

In [ ]:
def replace_with_constants(formula, existential_vars):
    """
    Replaces existential quantified variables with unique constant symbols.
    """
    used_constants = set()

    skolemized_formula = formula
    for var in existential_vars:
        constant_symbol = find_unused_constant(used_constants)
        used_constants.add(constant_symbol)

        skolemized_formula = skolemized_formula.replace('∃{}'.format(var), '')
        skolemized_formula = replace_variable_in_context(skolemized_formula, var, constant_symbol)

    return skolemized_formula

def find_unused_constant(used_constants):
    for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
        if letter not in used_constants:
            return letter
    raise ValueError("All uppercase letters used as constants.")



---



In [ ]:
def drop_universal_quantifiers(formula):
    """
    Eliminates universal quantifiers by removing both the quantifier and the character after it.
    """
    while '∀' in formula:
        index = formula.index('∀')
        formula = formula[:index] + formula[index+2:]  # Removing both '∀' and the character after it
    formula = formula.strip()
    return formula

In [ ]:
def convert_to_CNF(formula):
    """
    Converts a formula to Conjunctive Normal Form (CNF).
    """

    if '∨' not in formula or '∧' not in formula:
        return formula

    disjunctive_parts = formula.split('∨')

    cnf_components = []
    result = []

    for disjunctive_part in disjunctive_parts:
        if '∧' in disjunctive_part:
            conjunctive_parts = disjunctive_part.split('∧')
            for part in conjunctive_parts:
                cnf_components.append(part.strip())
        else:
          for conjunct_part in cnf_components:
            result.append(conjunct_part + ' ∨ ' + disjunctive_part.strip())

    cnf_formula = ' ∧ '.join(result)

    return cnf_formula

In [ ]:
def split_into_clauses(formula):
    """
    Splits a formula into clauses at conjunctions and returns a set of clauses.
    """
    # Split the formula into its conjunctions
    conjunctions = formula.split('∧')

    # Initialize a set to store the clauses
    clauses = set()

    # Iterate through the conjunctions
    for conjunction in conjunctions:
        # Add each conjunction as a clause to the set
        clauses.add(conjunction.strip())


    return rename_variables_last_step(clauses)

def rename_variables_last_step(clauses):
    """
    Rename variables in clauses to ensure no variable is used in more than one clause.
    """
    # Initialize a set to keep track of encountered variable names across clauses
    encountered_variables = set()

    # Iterate through each clause
    for clause in clauses:
        # Extract variables from the clause using regular expression
        variables = re.findall(r'\w+\((\w+)\)', clause)

        # Initialize a set to keep track of encountered variable names within the current clause
        encountered_variables_within_clause = set()

        # Iterate through variables
        for var in variables:
            # If the variable name has been encountered before in other clauses, rename it
            while var in encountered_variables:
                # Append a lowercase character to the variable name
                var = chr(ord(var) + 1) if var < 'z' else 'a'

            # Add the variable name to the set of encountered variable names across clauses
            encountered_variables.add(var)

            # Add the variable name to the set of encountered variable names within the current clause
            encountered_variables_within_clause.add(var)

            # Replace the variable in the clause with the renamed variable
            clause = clause.replace('(' + var + ')', '(' + var + ')')

        # Remove the variable names encountered within the current clause from the set of variable names across clauses
        encountered_variables -= encountered_variables_within_clause

    return clauses

# Example usage:
formula = 'Read(John) ∧ Wealthy(John) ∧ ~Happy(John) ∧ Smart(John)'
clauses = split_into_clauses(formula)
clauses = rename_variables_last_step(clauses)
print(clauses)

{'~Happy(John)', 'Wealthy(John)', 'Read(John)', 'Smart(John)'}


In [ ]:
def rename_variables_in_clauses(formula):
    """
    Renames variables in clauses so that each clause has a unique variable name.
    """
    return formula

In [ ]:
ready_for_resolution = []

for formula in KB:

  print("Original formula:", formula)

  formula = eliminate_implication(formula)
  print("After eliminating implication:", formula)

  formula = move_negation_inwards(formula)
  print("After moving negation inwards:", formula)

  formula = remove_double_not(formula)
  print("Removing double not:", formula)

  standardized_formula = standardize_variables(removed_formula)
  if standardized_formula != removed_formula:
    print("After standardizing variables:", standardized_formula)

  prenex_formula = prenex_form(standardized_formula)
  if prenex_formula != standardized_formula:
    print("After prenex form:", prenex_formula)

  formula = skolemize(formula)
  print("After Skolemization:", formula)

  formula = drop_universal_quantifiers(formula)
  print("After Droping Universal Quantifiers:", formula)

  cnf_formula = convert_to_CNF(formula)
  print('After Converting to CNF: ', cnf_formula)

  clauses = split_into_clauses(formula)
  print('Clauses: ', clauses)

  ready_for_resolution.extend(clauses)

  print ( "--------------------")

print('\nReady for Resolution: ', ready_for_resolution)

Original formula: ∀x (Read(x) → ~Stupid(x))
After eliminating implication: ∀x (~Read(x) ∨ ~Stupid(x))
After moving negation inwards: ∀x (~Read(x) ∨ ~Stupid(x))
Removing double not: ∀x (~Read(x) ∨ ~Stupid(x))
After Skolemization: ∀x (~Read(x) ∨ ~Stupid(x))
After Droping Universal Quantifiers: (~Read(x) ∨ ~Stupid(x))
After Converting to CNF:  (~Read(x) ∨ ~Stupid(x))
Clauses:  {'(~Read(x) ∨ ~Stupid(x))'}
--------------------
Original formula: Read(John) ∧ Wealthy(John)
After eliminating implication: Read(John) ∧ Wealthy(John)
After moving negation inwards: Read(John) ∧ Wealthy(John)
Removing double not: Read(John) ∧ Wealthy(John)
After Skolemization: Read(John) ∧ Wealthy(John)
After Droping Universal Quantifiers: Read(John) ∧ Wealthy(John)
After Converting to CNF:  Read(John) ∧ Wealthy(John)
Clauses:  {'Wealthy(John)', 'Read(John)'}
--------------------
Original formula: ∀x (Poor(x) → ~Wealthy(x))
After eliminating implication: ∀x (~Poor(x) ∨ ~Wealthy(x))
After moving negation inwards: ∀x

In [ ]:
def format_string(input_string):

    formatted_string = ''
    prev_char = ''
    for char in input_string:
        if prev_char == '(' and char == '(':
            continue
        if prev_char == ')' and char == ')':
            continue
        formatted_string += char
        prev_char = char

    while formatted_string and formatted_string[0] == '(':
        formatted_string = formatted_string[1:]

    return formatted_string

In [ ]:
ready_for_resolution = [format_string(cl) for cl in ready_for_resolution]
print(ready_for_resolution)

['~Read(x) ∨ ~Stupid(x)', 'Wealthy(John)', 'Read(John)', '~Poor(x) ∨ ~Wealthy(x)', 'Stupid(x) ∨ Smart(x)', 'Poor(x) ∨ ~Smart(x) ∨ Happy(x)', 'Exciting(x) ∨ ~Happy(x)']


## Applying Resolution: